In [2]:
import pandas as pd
import numpy as np
import ast
from pandas import json_normalize

from tqdm.notebook import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

# Часть 1. Предобработка данных

На данном этапе наша задача изучить данные и сформировать признаки, необходимые для работы модели

## Подготовка данных

### Загрузка даных

In [3]:
train = pd.read_parquet('~/kazan-exp/train.parquet')

In [4]:
test = pd.read_parquet('~/kazan-exp/test.parquet')

In [34]:
#text_fields = pd.read_csv('~/work/text_fields.csv')

In [162]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91120 entries, 0 to 91119
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     91120 non-null  int64  
 1   category_id    91120 non-null  int64  
 2   sale           91120 non-null  bool   
 3   shop_id        91120 non-null  int64  
 4   shop_title     91120 non-null  object 
 5   rating         91120 non-null  float64
 6   text_fields    91120 non-null  object 
 7   category_name  91120 non-null  object 
 8   level_1        91120 non-null  object 
 9   level_2        91120 non-null  object 
 10  level_3        91120 non-null  object 
 11  level_4        91120 non-null  object 
 12  level_5        91120 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(8)
memory usage: 8.4+ MB


In [5]:
train.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name
0,325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
4,1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...
5,1058275,12834,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...


In [6]:
test.head()

,product_id,sale,shop_id,shop_title,rating,text_fields
1,1997646,False,22758,Sky_Electronics,5.000000,"{""title"": ""Светодиодная лента Smart led Strip ..."
2,927375,False,17729,Di-Di Market,4.405941,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
3,1921513,False,54327,VisionStore,4.000000,"{""title"": ""Проводные наушники с микрофоном jac..."
4,1668662,False,15000,FORNAILS,5.000000,"{""title"": ""Декоративная табличка \""Правила кух..."
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,"{""title"": ""Подставка под ложку керамическая, п..."


In [7]:
train = train.reset_index(drop=True)

In [8]:
test = test.reset_index(drop=True)

In [9]:
#text_fields = text_fields.reset_index(drop=True)

Столбец text_fields представляет собой набор словарей с необходимой информацией для обработки текста.

### Разбивка стобца с описанием товара на отдельный датафрейм

**Функция для разбивки text_fields по столбцам**

Достанем из нее данные. Ресурсов компьютера не хватает для обработки всего столбца разом. Поэтому добавим параметр batch_size, в нем укажем сколько будет обработано элементов за цикл.

In [10]:
# с помощью функции формируем отдельный датафрейм где раскроем данные из словаря.
# Так даже удобнее анализировать, потом добавим их в общую таблицу
# batch_size нужен для того, чтобы обработать всю таблицу, если это делать не по кусочкам, то скорость падает до бесконечности.
# До этого df был глобальной переменной

def df_from_series_4(series, batch_size):
    
    df = pd.DataFrame(columns=pd.json_normalize(ast.literal_eval(series[0]), max_level=0).columns)
    batch_size = batch_size
    n_samples = series.shape[0]
    #global df
    
    for i in tqdm(range(0, series.shape[0], batch_size)):
        end = min(i + batch_size, n_samples)
        
        for j in series[i:end]:
            df = pd.concat([df, pd.json_normalize(ast.literal_eval(j), max_level=0)])
    df = df.reset_index(drop=True)        
    return df

In [11]:
# метод не оправдал ожиданий
# df = pd.DataFrame(columns=pd.json_normalize(ast.literal_eval(train['text_fields'][0]), max_level=0).columns)

Применим функцию распаковки на тренировочной и тестовой выборке

In [13]:
text_fields = df_from_series_4(train['text_fields'], 1000)
#df.to_csv('text_fields.csv')

  0%|          | 0/92 [00:00<?, ?it/s]

In [14]:
text_fields_test = df_from_series_4(test['text_fields'], 1000)

  0%|          | 0/17 [00:00<?, ?it/s]

Сохраним таблицы

In [15]:
text_fields.to_csv('text_fields')

In [16]:
text_fields_test.to_csv('text_fields')

In [18]:
text_fields_test.head()

,title,description,attributes,custom_characteristics,defined_characteristics,filters
0,"Светодиодная лента Smart led Strip Light, с пу...","<p>Светодиодная лента LED, 5 м, RGB (Цветная) ...","[Легкость управления с пульта, а так же смартф...",{},{},"{'Тип питания': ['От сети', 'От USB'], 'В комп..."
1,Стекло ПЛЕНКА керамик матовое Honor 50 lite 10...,"<p><span style=""color: rgb(63, 62, 62);"">Защит...",[Honor 50 ПЛЕНКА!!! КРАЯ КЛЕЮТСЯ МОГУТ НЕ ПРО...,"{'Модель': ['8a - Y6 2019', '9x - P Smart Z', ...",{},"{'Модель устройства': ['Honor 9X', 'Honor 20 P..."
2,"Проводные наушники с микрофоном jack 3.5, IOS,...",<p>Наушники проводные с микрофоном отличное ка...,[Возможность использования как гарнитуры для П...,{},{},"{'Разъем': ['3.5 мм'], 'Конструкция наушников'..."
3,"Декоративная табличка ""Правила кухни"", подстав...","<p>Декоративная табличка ""Правила кухни"" созда...",[],{},{},{'Бренд': ['FORNAILS']}
4,"Подставка под ложку керамическая, подложка ""Кл...",<p>Подложка керамическая с рисунком в подарочн...,"[Керамика., Размер: 255*90*30мм; ]","{'Модель': ['Клубника', 'Лимоны', 'Лаванда']}",{},"{'Бренд': ['Valleya', 'Valleya', 'Valleya']}"


Успешно

### Выделяем целевые признаки, которые будем предсказывать по цепочке

In [19]:
print(train[train['category_id'] == 12901]['category_name'])

34832    Все категории->Одежда->Мужская одежда->Толстов...
Name: category_name, dtype: object


In [20]:
train['category_id'].value_counts()

11937    6590
14922    3709
13651    1463
13143    1460
12980    1222
         ... 
12808       2
12901       1
11549       1
11875       1
12836       1
Name: category_id, Length: 874, dtype: int64

В целевом признаке имеются очень специфичные категории. Есть предположение, что если модель предварительно предскажет первую большую категорию category_name с высокой точностью, то за счет нового признака она будет точнее, это будем проверять дальше.

Идти понарастающей и последовательно предсказать каждый уровень категории товара вплоть до итогового кажется неплохой идеей.

In [21]:
# функция для применения apply
def split(text):
    return text.split('->')

In [22]:
split(train['category_name'][0])

['Все категории',
 'Электроника',
 'Смартфоны и телефоны',
 'Аксессуары и запчасти',
 'Зарядные устройства и кабели',
 'Кабели']

In [23]:
# сохраним виде таблицы
levels = pd.DataFrame(train['category_name'].progress_apply(split).to_list())

  0%|          | 0/91120 [00:00<?, ?it/s]

In [24]:
levels

,0,1,2,3,4,5
0,Все категории,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели
1,Все категории,Одежда,Женская одежда,Белье и купальники,Трусы,None
2,Все категории,Хобби и творчество,Рисование,"Краски, пигменты",None,None
3,Все категории,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,None
4,Все категории,Одежда,Женская одежда,Домашняя одежда,Пижамы,None
...,...,...,...,...,...,...
91115,Все категории,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Защитные стекла и пленки,Защитные стекла
91116,Все категории,Хобби и творчество,"Пазлы, мозаика и фреска",Алмазные мозаики,None,None
91117,Все категории,Товары для дома,Товары для праздников,Открытки и конверты,Открытки,None
91118,Все категории,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,None


In [26]:
# количество пропусков
pd.DataFrame(round(levels.isna().mean()*100,), columns=['percent']).style.background_gradient('coolwarm')

,percent
0,0.000000
1,0.000000
2,0.000000
3,1.000000
4,21.000000
5,85.000000


Выходит, что минимальное количество уровней в категориях товара - два, третий уровень имеет 1% пропусков. Предлагаю предсказать эти категории, дополнив итоговые модели полученными фичами.

Изучим пропуски третьего уровня:

In [27]:
levels[levels[3].isna()]

,0,1,2,3,4,5
106,Все категории,Хобби и творчество,Стикеры,None,None,None
169,Все категории,Хобби и творчество,Стикеры,None,None,None
572,Все категории,Хобби и творчество,Стикеры,None,None,None
603,Все категории,Хобби и творчество,Стикеры,None,None,None
781,Все категории,Хобби и творчество,Стикеры,None,None,None
...,...,...,...,...,...,...
91002,Все категории,Хобби и творчество,Стикеры,None,None,None
91049,Все категории,Хобби и творчество,Стикеры,None,None,None
91068,Все категории,Хобби и творчество,Стикеры,None,None,None
91080,Все категории,Хобби и творчество,Стикеры,None,None,None


In [28]:
levels[levels[3].isna()][2].value_counts()

Стикеры    1168
Name: 2, dtype: int64

По итогу, стикеры выбиваются из всех категорий, как товар с наименьшим количеством уровней. Категории товаров в небольшим количеством уровней продлим до последнего.

In [30]:
for i in range(levels.shape[1]):
    if i == 0:
        continue
    else:
        levels.loc[levels[i].isna(), i] = levels.loc[levels[i].isna(), i-1]

In [31]:
levels.columns = ['level_0', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5']

In [32]:
levels

,level_0,level_1,level_2,level_3,level_4,level_5
0,Все категории,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели
1,Все категории,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы
2,Все категории,Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты"
3,Все категории,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы
4,Все категории,Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы
...,...,...,...,...,...,...
91115,Все категории,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Защитные стекла и пленки,Защитные стекла
91116,Все категории,Хобби и творчество,"Пазлы, мозаика и фреска",Алмазные мозаики,Алмазные мозаики,Алмазные мозаики
91117,Все категории,Товары для дома,Товары для праздников,Открытки и конверты,Открытки,Открытки
91118,Все категории,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,Пульты ДУ


In [33]:
for i in levels:
    print('уровень -', i , '-', levels[i].value_counts().count(), 'категорий')

уровень - level_0 - 1 категорий
уровень - level_1 - 5 категорий
уровень - level_2 - 45 категорий
уровень - level_3 - 279 категорий
уровень - level_4 - 693 категорий
уровень - level_5 - 727 категорий


In [34]:
train['category_id'].value_counts().count()

874

Имеется несоответствие уникальных id и количества уникальных названий категорий товара с точки зрения дерева категорий KazanExpress. Вероятно, на некоторые категории приходится несколько индексов.

In [35]:
# дроп очевидной категории
levels = levels.drop(columns=['level_0'])

In [36]:
train = pd.concat([train, levels], axis=1)

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91120 entries, 0 to 91119
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     91120 non-null  int64  
 1   category_id    91120 non-null  int64  
 2   sale           91120 non-null  bool   
 3   shop_id        91120 non-null  int64  
 4   shop_title     91120 non-null  object 
 5   rating         91120 non-null  float64
 6   text_fields    91120 non-null  object 
 7   category_name  91120 non-null  object 
 8   level_1        91120 non-null  object 
 9   level_2        91120 non-null  object 
 10  level_3        91120 non-null  object 
 11  level_4        91120 non-null  object 
 12  level_5        91120 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(8)
memory usage: 8.4+ MB


In [38]:
train.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name,level_1,level_2,level_3,level_4,level_5
0,325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели
1,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы
2,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...,Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты"
3,1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы
4,1058275,12834,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...,Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы


Проверим как распределены индексы между категориями

In [39]:
cat_pt = train.pivot_table(index=['level_5'], values=['category_id'], aggfunc=lambda x: x.value_counts().count())

In [40]:
cat_pt.sort_values(by=['category_id'], ascending=False)[cat_pt['category_id'] > 1]

,category_id
level_5,
Комплекты,6
Аксессуары,5
Жилеты,5
Сапоги,4
Худи,4
...,...
Комбинезоны,2
Блузки,2
Комбинезоны и брюки утепленные,2


In [41]:
cat_pt.category_id.sum()

874

Сводная таблица подтверждает предположение, что одна и та же категория товара может находиться в разных тематических блоках, в связи с чем условные "аксессуары" могут иметь несколько разных id. Расмотрим их на примере трех топовых по количеству id категорий.

In [42]:
cat_pt = train.pivot_table(index=['level_1','level_2','level_3','level_4','level_5'],
                           values=['category_id'], aggfunc=lambda x: x.value_counts().count())

In [43]:
cat_pt[np.in1d(cat_pt.index.get_level_values(4),
            ['Аксессуары', 'Комплекты', 'Жилеты'])].sort_values(by=['level_5'])

category_id
level_1            level_2                     level_3                       level_4                       level_5                
Одежда             Женская одежда              Белье и купальники            Бюстгальтеры и аксессуары     Аксессуары            1
Товары для дома    Текстиль                    Шторы и карнизы               Аксессуары                    Аксессуары            1
                   Товары для праздников       Оформление праздника          Воздушные шары и аксессуары   Аксессуары            1
Хобби и творчество Музыкальные инструменты     Аксессуары                    Аксессуары                    Аксессуары            1
                   Настольные и карточные игры Аксессуары                    Аксессуары                    Аксессуары            1
Одежда             Детская одежда              Одежда для мальчиков          Джемперы, свитеры и кардиганы Жилеты                1
                   Женская одежда              Верхняя одежда                Жилеты                        Жилеты                1
                                               Джемперы, свитеры и кардиганы Жилеты                        Жилеты                1
                                               Жакеты и жилеты               Жилеты                        Жилеты                1
                   Мужская одежда              Верхняя одежда                Жилеты                        Жилеты                1
                   Детская одежда              Одежда для девочек            Костюмы и пиджаки             Комплекты             1
                                               Одежда для мальчиков          Белье и пляжная одежда        Комплекты             1
                                                                             Костюмы и пиджаки             Комплекты             1
                   Женская одежда              Домашняя одежда               Комплекты                     Комплекты             1
                                               Костюмы и комплекты           Комплекты                     Комплекты             1
                   Мужская одежда              Костюмы и комплекты           Комплекты                     Комплекты             1

Учитывая разнообразие схожих категорий, метод с дополнением признакового пространства путем предсказания более общих категорий товаров кажется наиболее целесообразным.

Далее необходимо перевести категории в числовой формат.
Предсказать их последовательно, основываясь на мешке слов.
Определить глубину предсказания на основании точности моделей.

### Лемматизация

In [44]:
text_fields.head()

,title,description,attributes,custom_characteristics,defined_characteristics,filters
0,Зарядный кабель Borofone BX1 Lightning для айф...,"<p><span style=""background-color: transparent;...","[Длина: 1м, Разъем: Lightning, Подерживает быс...",{},"{'Цвет': ['Черный', 'Белый']}","{'Цвет': ['Белый', 'Черный']}"
1,Трусы Sela,Трусы-слипы из эластичного бесшовного трикотаж...,[],{},{'Размер одежды': ['S']},"{'Бренд': ['Sela'], 'Размер одежды': ['S']}"
2,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12 цветов по 35 мл, БО...",<p>Гуашь высшего качества ЮНЛАНДИЯ поможет соз...,[],{},{},{'Бренд': ['Юнландия']}
3,Колба для кальяна Крафт (разные цвета),"<p><span style=""color: rgb(149, 151, 153);"">Ун...","[Материал: стекло, Внутренний диаметр: 45 мм ,...",{},"{'Цвет': ['Хром', 'Розовый', 'Желтый', 'Прозра...","{'Цвет': ['Прозрачный', 'Желтый', 'Синий', 'Ро..."
4,"Пижама женская, однотонная с шортами",<p>Лёгкая ткань! Комфортная посадка! Идеальная...,[],{},"{'Размер одежды': ['XXXL', 'L', 'XXL', 'XL'], ...","{'Цвет': ['Белый', 'Белый', 'Белый', 'Белый'],..."


In [45]:
text_fields.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91120 entries, 0 to 91119
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   title                    91120 non-null  object
 1   description              91120 non-null  object
 2   attributes               91120 non-null  object
 3   custom_characteristics   91120 non-null  object
 4   defined_characteristics  91120 non-null  object
 5   filters                  91120 non-null  object
dtypes: object(6)
memory usage: 4.2+ MB


В качестве набора признаков будем использовать информацию о названии товара.

In [46]:
#tfhead = text_fields.head(1000)

In [47]:
#tfhead

Сформируем таблицу признаков. В дальнейшем мы исследуем два пути. Первый - предскажем сразу конечную категорию товара, второй - попробуем предсказать всё дерево.

In [48]:
X = text_fields['title']
#+ ' ' + tfhead['description'] # Описание запутывает модель

In [49]:
X = pd.DataFrame(X)

In [51]:
X[['level_1','level_2','level_3','level_4','level_5']] =  train[['level_1','level_2','level_3','level_4','level_5']]
#about.columns = ['about', 'target']

In [52]:
X

,title,level_1,level_2,level_3,level_4,level_5
0,Зарядный кабель Borofone BX1 Lightning для айф...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели
1,Трусы Sela,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы
2,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12 цветов по 35 мл, БО...",Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты"
3,Колба для кальяна Крафт (разные цвета),Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы
4,"Пижама женская, однотонная с шортами",Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы
...,...,...,...,...,...,...
91115,Прочное стекло 2D на Samsung Galaxy J5 Prime,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Защитные стекла и пленки,Защитные стекла
91116,"Алмазная мозаика ""Ромашки"" 40*50см на подрамнике",Хобби и творчество,"Пазлы, мозаика и фреска",Алмазные мозаики,Алмазные мозаики,Алмазные мозаики
91117,"Открытка ""Вместе навсегда"" в крафтовом конверт...",Товары для дома,Товары для праздников,Открытки и конверты,Открытки,Открытки
91118,Пульт K10B-C1 для Rolsen,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,Пульты ДУ


In [55]:
X_test = pd.DataFrame(text_fields_test['title'])

### **Чистка от мусора и дубликатов.**

Для собственного удобства импортирую нужные библиотеки в этом разделе

In [56]:
import pandas as pd
from pymystem3 import Mystem 
import re 
m = Mystem() # класс Mystem для лемматизации

In [95]:
corpus = list(X['title'])

corpus_2 = list(X_test['title']) # второй раз производим манипуляцию на тестовой выборке

In [96]:
def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text

In [117]:
def clear_text(text):
    text = re.sub(r'[^а-яА-ЯёЁ]', ' ', text)
    return " ".join(text.split())

print("Исходный текст:", corpus_2[2])
print("Очищенный и лемматизированный текст:", lemmatize(clear_text(corpus_2[2])))

Исходный текст: Проводные наушники с микрофоном jack 3.5, IOS, Android
Очищенный и лемматизированный текст: проводной наушник с микрофон



Функции работают, однако проблема в том, что в списке могут быть названия товаров на английском: бренды, названия устройств и тд. Поэтому перепишем функцию очистки 

In [122]:
# удаляет цифры, знаки и выравнивает размер текта
def clear_text(text):
    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', text)
    text = ' '.join(word for word in text.split() if len(word)>3)
    return " ".join(text.split()).lower()

print("Исходный текст:", corpus_2[2])
print("Очищенный и лемматизированный текст:", lemmatize(clear_text(corpus_2[2])))

Исходный текст: Проводные наушники с микрофоном jack 3.5, IOS, Android
Очищенный и лемматизированный текст: проводной наушник микрофон jack android



Функция работает, теперь в названии большую часть составляет полезная информация

In [45]:
# # Очитска текста от лишних прилагательных
# def clear_text_2(text):
#     text = re.sub(r'(\b\w{1,}\Bой|\b\w{1,}\Bий|\b\w{1,}\Bый|\b\w{1,}\Bого|\b\w{1,}\Bего| \
#     \b\w{1,}\Bому|\b\w{1,}\Bему|\b\w{1,}\Bим|\b\w{1,}\Bым|\b\w{1,}\Bом|\b\w{1,}\Bое|\b\w{1,}\Bее|\b\w{1,}\Bое|\b\w{1,}\Bее| \
#     \b\w{1,}\Bая|\b\w{1,}\Bяя|\b\w{1,}\Bую|\b\w{1,}\Bюю|\b\w{1,}\Bие|\b\w{1,}\Bые|\b\w{1,}\Bих|\b\w{1,}\Bых|\b\w{1,} \
#     \b\w{1,}\Bими|\b\w{1,}\Bыми|\b\w{1,2}\b)', ' ', text)
    
#     return " ".join(text.split())

Имеются незапущенные ячейки кода, так как проект был приведен в порядок после формирования нужных таблиц

In [166]:
X['lemm'] = X['title'].progress_apply(clear_text)

  0%|          | 0/91120 [00:00<?, ?it/s]

In [163]:
#X['lemm'] = X['lemm'].progress_apply(lemmatize)

Ожидается очень долгая обработка данных, но при этом отдельные предложения PyMystem3 лемматизирует мгновенно. Поиск информации показал, что "PyMystem3 использует для обработки внешний исполняемый файл mystem.exe. Видимо, такая архитектура на частых маленьких вызовах обходится очень дорого." поэтому объединим наши названия в список крупных текстов. Разделителем укажем какое-нибудь необычное слово, например " оранжеваяобезьяна "

In [178]:
def lemmatize_3(list):
    
    # блок объединения
    join_lemm = []

    for i in tqdm(range(0, len(list), 1000)):
        end = min(i + 1000, len(list)) #позволяет не выйти за пределы списка
        join_lemm.append(' оранжеваяобезьяна '.join(list[i:end])) 
    
    # блок лемматизации
    lemmatized = []
        
    for i in tqdm(range(len(join_lemm))):
        element = lemmatize(join_lemm[i])
        element = element.split(' оранжеваяобезьяна ')
        
        for g in element:
            lemmatized.append(g)
                   
    return lemmatized

In [168]:
X['lemm'] = lemmatize_3(X['lemm'].to_list())

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

Таким образом, мы сократили время обработки тексты с 14 часов до 1 минуты. Это очень радует.

In [173]:
X.tail()

,title,level_1,level_2,level_3,level_4,level_5,lemm
91115,Прочное стекло 2D на Samsung Galaxy J5 Prime,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Защитные стекла и пленки,Защитные стекла,прочный стекло samsung galaxy prime
91116,"Алмазная мозаика ""Ромашки"" 40*50см на подрамнике",Хобби и творчество,"Пазлы, мозаика и фреска",Алмазные мозаики,Алмазные мозаики,Алмазные мозаики,алмазный мозаика ромашка подрамник
91117,"Открытка ""Вместе навсегда"" в крафтовом конверт...",Товары для дома,Товары для праздников,Открытки и конверты,Открытки,Открытки,открытка вместе навсегда крафтовый конверт фев...
91118,Пульт K10B-C1 для Rolsen,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,Пульты ДУ,пульт rolsen
91119,Пульт DS300A/DC300A/DN300 для цифровой пристав...,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,Пульты ДУ,пульт цифровой приставка\n


In [174]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91120 entries, 0 to 91119
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    91120 non-null  object
 1   level_1  91120 non-null  object
 2   level_2  91120 non-null  object
 3   level_3  91120 non-null  object
 4   level_4  91120 non-null  object
 5   level_5  91120 non-null  object
 6   lemm     91120 non-null  object
dtypes: object(7)
memory usage: 4.9+ MB


Далее разберемся с тестовой выборкой

In [180]:
X_test['lemm'] = lemmatize_3(X_test['lemm'].to_list())

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [180]:
X_test['lemm'] = lemmatize_3(X_test['lemm'].to_list())

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [176]:
X_test['lemm'] = X_test['title'].progress_apply(clear_text)

  0%|          | 0/16860 [00:00<?, ?it/s]

In [181]:
X_test.head()

,title,lemm
0,"Светодиодная лента Smart led Strip Light, с пу...",светодиодный лента smart strip light пульт мет...
1,Стекло ПЛЕНКА керамик матовое Honor 50 lite 10...,стекло пленка керамика матовый honor lite psma...
2,"Проводные наушники с микрофоном jack 3.5, IOS,...",проводной наушник микрофон jack android
3,"Декоративная табличка ""Правила кухни"", подстав...",декоративный табличка правило кухня подставка ...
4,"Подставка под ложку керамическая, подложка ""Кл...",подставка ложка керамический подложка клубника...


In [182]:
X.to_csv('train_lemm.csv')

In [183]:
X_test.to_csv('test_lemm.csv')

По итогу, нами сформировано 2 таблицы, которые содержат лемматизированные названия товаров, в следующей части, будет проведена работа 
